In [41]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil
import re

In [4]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp_2022/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'

In [3]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [2]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [32]:
def clean_tel(row, col_name):
    if pd.isna(row[col_name]):
        return np.nan
    s = str(row[col_name])
    ans = ''
    for c in s:
        if c == '.':
            break
        if c.isalnum():
            ans += c
    return ans[1:]

In [33]:
base = pd.read_excel(path_to_data + 'ppl_finder.xlsx', sheet_name='base_data').rename(columns={'СНИЛС': 'snils', 'ФИО': 'fio', 'ТелМобильный': 'tel', 'АдресРегистрации': 'addr'})
base['tel'] = base.apply(lambda row: clean_tel(row, 'tel'), axis=1)
base['tel']

0         9060481455
1         9060481455
2         9266890810
3         9998324367
4         9157820404
             ...    
218944    9067802804
218945    9082894014
218946    9296209167
218947    9037007737
218948    9259400233
Name: tel, Length: 218949, dtype: object

In [35]:
search = pd.read_excel(path_to_data + 'ppl_finder.xlsx', sheet_name='searcher').rename(columns={'ФИО': 'fio', 'Телефон': 'tel'})
search['tel'] = search.apply(lambda row: clean_tel(row, 'tel'), axis=1)
search['tel']

0         9250360351
1         9167197062
2         9261333513
3         9168361266
4         9163788730
             ...    
120080    9998155795
120081    9055054293
120082    9164151425
120083    9266472652
120084    9856193387
Name: tel, Length: 120085, dtype: object

In [36]:
ans1 = pd.merge(search['fio'], base, how='left', on='fio').drop_duplicates()
ans1

,fio,snils,tel,addr
0,Абызова Татьяна Николаевна,062-969-796 25,9250360351,"РОССИЯ, 117405, Москва г, Варшавское ш, дом № ..."
1,Акопджанова Кира Вадимовна,029-095-312 56,9167197062,"РОССИЯ, 125222, Москва г, Волоцкой пер., дом №..."
3,Аббасова Парвана Теймур Кызы,138-524-894 90,9261333513,"РОССИЯ, 115193, Москва г, Петра Романова ул, д..."
5,Авдеева Виктория Михайловна,038-767-061 92,9168361266,"115597, Москва г, Ясеневая Ул, дом № 42, корпу..."
6,Авилова Оксана Григорьевна,NaN,NaN,NaN
...,...,...,...,...
161413,Ярославцев Роман Олегович,NaN,NaN,NaN
161414,Ярошевич София Григорьевна,188-047-801 00,9055054293,"РОССИЯ, 115573, Москва г, Шипиловская ул, дом ..."
161415,Ярыгина Елена Сергеевна,NaN,NaN,NaN
161416,Ярюкина Алена Игоревна,151-619-841 65,9266472652,"РОССИЯ, 127282, Москва г, Студёный проезд, дом..."


In [37]:
search.dropna(subset='tel')

,fio,tel
0,Абызова Татьяна Николаевна,9250360351
1,Акопджанова Кира Вадимовна,9167197062
2,Аббасова Парвана Теймур Кызы,9261333513
3,Авдеева Виктория Михайловна,9168361266
4,Авилова Оксана Григорьевна,9163788730
...,...,...
120080,Ярославцев Роман Олегович,9998155795
120081,Ярошевич София Григорьевна,9055054293
120082,Ярыгина Елена Сергеевна,9164151425
120083,Ярюкина Алена Игоревна,9266472652


In [38]:
ans2 = pd.merge(search['tel'], base, how='left', on='tel').drop_duplicates()
ans2

,tel,snils,fio,addr
0,9250360351,062-969-796 25,Абызова Татьяна Николаевна,"РОССИЯ, 117405, Москва г, Варшавское ш, дом № ..."
1,9167197062,029-095-312 56,Акопджанова Кира Вадимовна,"РОССИЯ, 125222, Москва г, Волоцкой пер., дом №..."
3,9261333513,138-524-894 90,Аббасова Парвана Теймур Кызы,"РОССИЯ, 115193, Москва г, Петра Романова ул, д..."
5,9168361266,038-767-061 92,Авдеева Виктория Михайловна,"115597, Москва г, Ясеневая Ул, дом № 42, корпу..."
6,9163788730,NaN,NaN,NaN
...,...,...,...,...
11913933,9998155795,NaN,NaN,NaN
11913934,9055054293,188-047-801 00,Ярошевич София Григорьевна,"РОССИЯ, 115573, Москва г, Шипиловская ул, дом ..."
11913935,9164151425,NaN,NaN,NaN
11913936,9266472652,151-619-841 65,Ярюкина Алена Игоревна,"РОССИЯ, 127282, Москва г, Студёный проезд, дом..."


In [70]:
ans = pd.concat([ans1[['snils', 'fio', 'tel', 'addr']], ans2[['snils', 'fio', 'tel', 'addr']]]).drop_duplicates().dropna()
ans

,snils,fio,tel,addr
0,062-969-796 25,Абызова Татьяна Николаевна,9250360351,"РОССИЯ, 117405, Москва г, Варшавское ш, дом № ..."
1,029-095-312 56,Акопджанова Кира Вадимовна,9167197062,"РОССИЯ, 125222, Москва г, Волоцкой пер., дом №..."
3,138-524-894 90,Аббасова Парвана Теймур Кызы,9261333513,"РОССИЯ, 115193, Москва г, Петра Романова ул, д..."
5,038-767-061 92,Авдеева Виктория Михайловна,9168361266,"115597, Москва г, Ясеневая Ул, дом № 42, корпу..."
7,106-528-592 57,Абубакирова Эльвира Юрьевна,9295244745,"РОССИЯ, 117246, Москва г, Херсонская ул, дом №..."
...,...,...,...,...
11827377,043-923-744 61,Салманов Назим Нафе оглы,9035428645,"РОССИЯ, 125635, Москва г, ул Весенняя, д. 3, к..."
11837241,171-258-929 80,Шабрина Анастасия Ильинична,9150471751,"РОССИЯ, 109544, Москва г, Рабочая ул, дом 8, к..."
11861888,154-540-279 55,Козлова Алена Валериевна,9998384748,"РОССИЯ, 127221, Москва г, Шокальского проезд, ..."
11871936,055-322-174 31,Лазаревич Ирина Юрьевна,9165570932,"124498, Москва г, Зеленоград г, корпус № 426, ..."


In [47]:
base = base.dropna()

In [71]:
ans = pd.concat([ans, base[base['addr'].str.contains('Москва г')][['snils', 'fio', 'addr']]]).drop_duplicates().dropna()
ans

,snils,fio,tel,addr
0,062-969-796 25,Абызова Татьяна Николаевна,9250360351,"РОССИЯ, 117405, Москва г, Варшавское ш, дом № ..."
1,029-095-312 56,Акопджанова Кира Вадимовна,9167197062,"РОССИЯ, 125222, Москва г, Волоцкой пер., дом №..."
3,138-524-894 90,Аббасова Парвана Теймур Кызы,9261333513,"РОССИЯ, 115193, Москва г, Петра Романова ул, д..."
5,038-767-061 92,Авдеева Виктория Михайловна,9168361266,"115597, Москва г, Ясеневая Ул, дом № 42, корпу..."
7,106-528-592 57,Абубакирова Эльвира Юрьевна,9295244745,"РОССИЯ, 117246, Москва г, Херсонская ул, дом №..."
...,...,...,...,...
11827377,043-923-744 61,Салманов Назим Нафе оглы,9035428645,"РОССИЯ, 125635, Москва г, ул Весенняя, д. 3, к..."
11837241,171-258-929 80,Шабрина Анастасия Ильинична,9150471751,"РОССИЯ, 109544, Москва г, Рабочая ул, дом 8, к..."
11861888,154-540-279 55,Козлова Алена Валериевна,9998384748,"РОССИЯ, 127221, Москва г, Шокальского проезд, ..."
11871936,055-322-174 31,Лазаревич Ирина Юрьевна,9165570932,"124498, Москва г, Зеленоград г, корпус № 426, ..."


In [51]:
df = pd.read_excel('C:/Users/PetukhovMD/Desktop/2022/март/ДОНМ_Кадровые_паспортные данные.xlsx')

In [57]:
df = df.rename(columns={'Организация': 'org', 'ДР': 'bd', 'СНИЛС': 'snils', 'Состояние': 'sost'})

In [72]:
ans = pd.merge(ans, df[df.ВидЗанятости == 'Основное место работы'][['snils', 'org', 'bd', 'sost']], how='left', on='snils').drop_duplicates()

In [73]:
ans

,snils,fio,tel,addr,org,bd,sost
0,062-969-796 25,Абызова Татьяна Николаевна,9250360351,"РОССИЯ, 117405, Москва г, Варшавское ш, дом № ...",ГБОУ ШКОЛА № 924,21.03.1982,Работа
1,029-095-312 56,Акопджанова Кира Вадимовна,9167197062,"РОССИЯ, 125222, Москва г, Волоцкой пер., дом №...",ГБОУ Школа №1544,22.06.1980,Работа
2,138-524-894 90,Аббасова Парвана Теймур Кызы,9261333513,"РОССИЯ, 115193, Москва г, Петра Романова ул, д...",ГБОУ Школа имени Маяковского,24.03.1970,Работа
4,038-767-061 92,Авдеева Виктория Михайловна,9168361266,"115597, Москва г, Ясеневая Ул, дом № 42, корпу...",ГБОУ Школа № 949,29.05.1979,Работа
5,106-528-592 57,Абубакирова Эльвира Юрьевна,9295244745,"РОССИЯ, 117246, Москва г, Херсонская ул, дом №...",ГБОУ Школа № 1948,09.11.1975,Работа
...,...,...,...,...,...,...,...
123329,043-923-744 61,Салманов Назим Нафе оглы,9035428645,"РОССИЯ, 125635, Москва г, ул Весенняя, д. 3, к...",ГАОУ ВО МГПУ,21.01.1954,Работа
123330,171-258-929 80,Шабрина Анастасия Ильинична,9150471751,"РОССИЯ, 109544, Москва г, Рабочая ул, дом 8, к...",ГАОУ ВО МГПУ,27.04.1996,Работа
123331,154-540-279 55,Козлова Алена Валериевна,9998384748,"РОССИЯ, 127221, Москва г, Шокальского проезд, ...",ГАОУ ВО МГПУ,06.08.1991,Работа
123332,055-322-174 31,Лазаревич Ирина Юрьевна,9165570932,"124498, Москва г, Зеленоград г, корпус № 426, ...",ГАОУ ВО МГПУ,17.07.1962,Работа


In [74]:
print_df(ans[['org', 'snils', 'fio', 'bd', 'tel', 'sost', 'addr']], 'Люди с пропиской в мск')